In [ ]:
from utils import *

from random_cycles import gen_random_cycles
from local_search import local_search_steepest, local_search_greedy


%load_ext autoreload
%autoreload 2

### Utils

In [ ]:
from copy import deepcopy

### Hybrid Evolutionary Algorithm

In [ ]:
def gen_initial_population(data, distance_matrix, size):
    start_time = time.time()
    population = []
    for _ in range(size):
        randoms = gen_random_cycles(get_nodes(data))
        local_searched = local_search_steepest(*randoms, distance_matrix, data)
        population.append(local_searched)
    print("Initial population generated in: ", time.time() - start_time)
    return population

In [ ]:
def recombine(parent1, parent2, distance_matrix): # crossing over

    edges_in_parent_2 = set()
    for c in parent2:
        for i in range(len(c)):
            edges_in_parent_2.add((c[i], c[(i+1) % len(c)]))

    print("Krawędzie w parent2: ", edges_in_parent_2)

    cycle1 = deepcopy(parent1[0])
    cycle2 = deepcopy(parent1[1])

    print("Długość cyklu 1: ", len(cycle1))
    print("Długość cyklu 2: ", len(cycle2))
    print()

    cycle1_marked = [False] * len(cycle1)
    cycle2_marked = [False] * len(cycle2)

    for i in range(len(cycle1)):
        if (cycle1[i], cycle1[(i+1) % len(cycle1)]) in edges_in_parent_2:
            cycle1_marked[i] = True
            cycle1_marked[(i+1) % len(cycle1)] = True
    
    for i in range(len(cycle2)):
        if (cycle2[i], cycle2[(i+1) % len(cycle2)]) in edges_in_parent_2:
            cycle2_marked[i] = True
            cycle2_marked[(i+1) % len(cycle2)] = True

    free_nodes = []
    for i in range(len(cycle1)):
        if not cycle1_marked[i]:
            free_nodes.append(cycle1[i])
    for i in range(len(cycle2)):
        if not cycle2_marked[i]:
            free_nodes.append(cycle2[i])

    # remove False nodes
    cycle1 = [cycle1[i] for i in range(len(cycle1)) if cycle1_marked[i]]
    cycle2 = [cycle2[i] for i in range(len(cycle2)) if cycle2_marked[i]]

    if len(cycle1) == 0 or len(cycle2) == 0:
        return parent1[0], parent1[1]

    print("Po niszczeniu")
    print("Długość cyklu 1: ", len(cycle1))
    print("Długość cyklu 2: ", len(cycle2))
    print("Długość free_nodes: ", len(free_nodes))
    print("______")

    while len(free_nodes) > 0:
        if len(cycle1) < 100:
            best_update1 = float('inf')
            best_node1 = None
            best_position1 = -1
            
            for node in free_nodes:
                for i in range(len(cycle1)):
                    distance_update = distance_matrix[cycle1[i-1]][node] + distance_matrix[node][cycle1[i]] - distance_matrix[cycle1[i-1]][cycle1[i]]
                    if distance_update < best_update1:
                        best_update1 = distance_update
                        best_node1 = node
                        best_position1 = i
            
            if best_node1 is not None:
                cycle1.insert(best_position1, best_node1)
                free_nodes.remove(best_node1)

        if len(cycle2) < 100:
            best_update2 = float('inf')
            best_node2 = None
            best_position2 = -1
            
            for node in free_nodes:
                for i in range(len(cycle2)):
                    distance_update = distance_matrix[cycle2[i-1]][node] + distance_matrix[node][cycle2[i]] - distance_matrix[cycle2[i-1]][cycle2[i]]
                    if distance_update < best_update2:
                        best_update2 = distance_update
                        best_node2 = node
                        best_position2 = i
            
            if best_node2 is not None:
                cycle2.insert(best_position2, best_node2)
                free_nodes.remove(best_node2)

    print("Po odbudowie")
    print("Długość cyklu 1: ", len(cycle1))
    print("Długość cyklu 2: ", len(cycle2))
    print("Długość free_nodes: ", len(free_nodes))
    print("______")

    return cycle1, cycle2

In [ ]:
def hybrid_evolutionary_algorithm(distance_matrix, data, population_size, num_generations, local=False):
    population = gen_initial_population(data, distance_matrix, population_size)
    
    for _ in range(num_generations):
        
        parent_indices = np.random.choice(len(population), 2, replace=False)
        parent1, parent2 = population[parent_indices[0]], population[parent_indices[1]]
                
        offspring = recombine(parent1, parent2, distance_matrix)
        if local:
            offspring = local_search_steepest(*offspring, distance_matrix, data)
        
        worst_index = np.argmax([calculate_cycles_length(*ind, distance_matrix) for ind in population])
        worst_length = calculate_cycles_length(*population[worst_index], distance_matrix)

        offspring_length = calculate_cycles_length(*offspring, distance_matrix)
        
        if offspring_length < worst_length and is_diverse(offspring, population, distance_matrix):
            population[worst_index] = offspring
    
    best_index = np.argmin([calculate_cycles_length(*ind, distance_matrix) for ind in population])
    return population[best_index]



def is_diverse(individual, population,  distance_matrix, threshold=0.1):
    individual_length = calculate_cycles_length(*individual, distance_matrix)
    for other in population:
        other_length = calculate_cycles_length(*other, distance_matrix)
        if abs(individual_length - other_length) < threshold * individual_length:
            return False
    return True

### Run

In [ ]:
times = []
lengths = []
cycles = []


for filename in ['data/kroB200.tsp', 'data/kroA200.tsp']:

    file = filename.split('/')[-1]
    data = read_data_file(filename)
    distance_matrix = calculate_distance_matrix(data)

    for i in range(10):
        for local in [True, False]:
            print(f"Processing: {i} {local} {filename}")
            start = time.time()

            cycle1, cycle2 = hybrid_evolutionary_algorithm(
                distance_matrix,
                data,
                21,
                100,
                local=local)
            
            time_taken = time.time() - start
            length = calculate_cycles_length(cycle1, cycle2, distance_matrix)

            times.append((file, local, time_taken))
            lengths.append((file, local, length))
            cycles.append((file, local, cycle1, cycle2))



In [ ]:
times_df = pd.DataFrame(times, columns=["Instance", "Method", "Time"])
times_df

In [ ]:
lengths_df = pd.DataFrame(lengths, columns=["Instance", "Method", "Length"])
lengths_df

In [ ]:
# Grupowanie danych i obliczanie statystyk
time_stats = times_df.groupby(["Instance", "Method"])["Time"].agg(['min', 'mean', 'max']).reset_index()
length_stats = lengths_df.groupby(["Instance", "Method"])["Length"].agg(['min', 'mean', 'max']).reset_index()

# Łączenie statystyk w jeden DataFrame
stats_df = pd.merge(time_stats, length_stats, on=["Instance", "Method"], suffixes=("_time", "_length"))
stats_df

In [ ]:
cycles_df = pd.DataFrame(cycles, columns=["Instance", "Method", "Cycle1", "Cycle2"])
def calculate_length(row):
    data = read_data_file("data/" + row['Instance'])
    distance_matrix = calculate_distance_matrix(data)
    return calculate_cycles_length(row['Cycle1'], row['Cycle2'], distance_matrix)

cycles_df = pd.DataFrame(cycles, columns=["Instance", "Method", "Cycle1", "Cycle2"])
cycles_df['Cycle_Length'] = cycles_df.apply(calculate_length, axis=1)
cycles_df

In [ ]:
# Group by 'Instance' and 'Method' and find the index of the minimum 'Cycle_Length'
min_cycle_indices = cycles_df.groupby(['Instance', 'Method'])['Cycle_Length'].idxmin()

# Use the indices to get the rows with the minimum 'Cycle_Length'
min_cycles_df = cycles_df.loc[min_cycle_indices]

min_cycles_df

# if dir plots does not exist create it
if not os.path.exists('plots'):
    os.makedirs('plots')

for i, row in min_cycles_df.iterrows():
    plot_cycles(row['Cycle1'], row['Cycle2'], read_data_file("data/" + row['Instance']), f"plots/plot_{row['Instance'].replace('/', '')}_{row['Method']}.png")